In [1]:
import numpy as np
import pandas as pd
import pickle
from urllib.request import urlopen
import json
from sklearn.preprocessing import OneHotEncoder

In [2]:
historical_df = pd.read_pickle("prices_fundamentals.pkl")

In [3]:
historical_df

,ticker,date,close,volume,calendardate,accoci,assets,assetsavg,assetsc,assetsnc,...,volume_change_three_year_avg,divyield,marketcap,ev,evebit,pb,pe,peg1,peg2,peg3
248,A,2018-03-29,66.90,2711285.0,2017-12-31,-269000000.0,8.698000e+09,8.350250e+09,4.397000e+09,4.301000e+09,...,0.080627,0.008146,2.157368e+10,2.083168e+10,18.435118,4.770828,107.903226,-178.324278,-178.324278,-178.324278
249,A,2018-04-02,64.43,3085753.0,2018-03-31,-310000000.0,8.784000e+09,8.542250e+09,4.525000e+09,4.259000e+09,...,0.089415,0.008723,2.061452e+10,1.971852e+10,16.984079,4.468787,85.906667,-147.268571,-186.578542,-164.609213
250,A,2018-04-03,65.42,2286132.0,2018-03-31,-310000000.0,8.784000e+09,8.542250e+09,4.525000e+09,4.259000e+09,...,0.073329,0.008591,2.093127e+10,2.003527e+10,17.256906,4.537452,87.226667,-149.531429,-189.445417,-167.138518
251,A,2018-04-04,65.45,3702954.0,2018-03-31,-310000000.0,8.784000e+09,8.542250e+09,4.525000e+09,4.259000e+09,...,0.083420,0.008587,2.094087e+10,2.004487e+10,17.265174,4.539533,87.266667,-149.600000,-189.532292,-167.215164
252,A,2018-04-05,65.68,2118121.0,2018-03-31,-310000000.0,8.784000e+09,8.542250e+09,4.525000e+09,4.259000e+09,...,0.066764,0.008557,2.101446e+10,2.011846e+10,17.328558,4.555486,87.573333,-150.125714,-190.198333,-167.802780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112076,ZYXI,2021-02-23,18.07,646106.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.228709,0.000000,6.297301e+08,5.975411e+08,50.871883,11.076073,66.925926,-970.425926,-970.425926,384.824074
3112077,ZYXI,2021-02-24,18.42,315802.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.224603,0.000000,6.419275e+08,6.097385e+08,51.910306,11.290607,68.222222,-989.222222,-989.222222,392.277778
3112078,ZYXI,2021-02-25,17.42,388872.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.225354,0.000000,6.070780e+08,5.748890e+08,48.943383,10.677653,64.518519,-935.518519,-935.518519,370.981481
3112079,ZYXI,2021-02-26,14.52,2337972.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.234896,0.000000,5.060145e+08,4.738255e+08,40.339305,8.900088,53.777778,-779.777778,-779.777778,309.222222


In [4]:
import datetime
from datetime import timedelta, date

In [5]:
last_date = np.sort(historical_df.date.unique())[-1]

In [6]:
last_date = datetime.datetime.utcfromtimestamp(last_date.tolist()/1e9)

In [7]:
sdate = last_date
edate = datetime.datetime.now() 
delta = edate - sdate     

In [8]:
days = []

for i in range(delta.days):
    day = str((sdate + timedelta(days=i)).date())
    days.append(day)

dates = ", ".join(days)

In [9]:
#get market data
import quandl
quandl.ApiConfig.api_key = "PCQqdJWPc-fYUJQ3JbGA"

In [10]:
date_today = edate.date()
date_today

datetime.date(2021, 3, 17)

In [11]:
if date_today.month <= 3:
    calendar_dates = str(date_today.year-1)+"-12-31" + ", " + str(date_today.year-1)+"-09-30"
elif date_today.month <= 6 and date_today.month > 3:
    calendar_dates = str(date_today.year)+"-03-31" + ", " + str(date_today.year-1)+"-12-31"
elif date_today.month <= 9 and date_today.month > 6:
    calendar_dates = str(date_today.year)+"-03-31" + ", " + str(date_today.year)+"-06-30"
else:
    calendar_dates = str(date_today.year)+"-09-30" + ", " + str(date_today.year)+"-06-30"

In [12]:
with open('final_tickers.pkl', 'rb') as handle:
    final_tickers = pickle.load(handle)

In [13]:
final_tickers

['A - Agilent Technologies Inc',
 'AA - Alcoa Corp',
 'AAL - American Airlines Group Inc',
 'AAOI - Applied Optoelectronics Inc',
 'AAON - Aaon Inc',
 'AAP - Advance Auto Parts Inc',
 'AAPL - Apple Inc',
 'AAT - American Assets Trust Inc',
 'AAWW - Atlas Air Worldwide Holdings Inc',
 'ABB - Abb Ltd',
 'ABBV - AbbVie Inc',
 'ABC - Amerisourcebergen Corp',
 'ABCB - Ameris Bancorp',
 'ABEO - Abeona Therapeutics Inc',
 'ABG - Asbury Automotive Group Inc',
 'ABM - Abm Industries Inc',
 'ABMD - Abiomed Inc',
 'ABR - Arbor Realty Trust Inc',
 'ABT - Abbott Laboratories',
 'ABTX - Allegiance Bancshares Inc',
 'ABUS - Arbutus Biopharma Corp',
 'ACAD - Acadia Pharmaceuticals Inc',
 'ACBI - Atlantic Capital Bancshares Inc',
 'ACC - American Campus Communities Inc',
 'ACCO - ACCO BRANDS Corp',
 'ACGL - Arch Capital Group Ltd',
 'ACHC - Acadia Healthcare Company Inc',
 'ACIA - Acacia Communications Inc',
 'ACIW - Aci Worldwide Inc',
 'ACLS - Axcelis Technologies Inc',
 'ACM - Aecom',
 'ACMR - ACM R

In [14]:
final_ticks = []
for tick in final_tickers:
    final_ticks.append(tick.split(" - ")[0])

In [16]:
sf1 = pd.DataFrame()

In [17]:
for tick in final_ticks:
    sf1_ticker = quandl.get_table('SHARADAR/SF1', calendardate=calendar_dates, ticker=tick)
    sf1 = sf1.append(sf1_ticker)

In [18]:
sf1

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
0,A,MRY,2020-12-31,2020-10-31,2020-10-31,2021-03-02,-522000000,9627000000,9532250000.0,3415000000,...,308309635,309000000,312000000,17.278,5194000000,0,123000000,0,16.809,1948000000
1,A,MRT,2020-12-31,2021-01-31,2021-01-31,2021-03-02,-469000000,9674000000,9575500000.0,3483000000,...,306849526,306000000,309000000,18.072,5269000000,0,125000000,0,17.219,1796000000
2,A,MRT,2020-09-30,2020-10-31,2020-10-31,2021-03-02,-522000000,9627000000,9532250000.0,3415000000,...,308309635,309000000,312000000,17.278,5194000000,0,123000000,0,16.809,1948000000
3,A,MRQ,2020-12-31,2021-01-31,2021-01-31,2021-03-02,-469000000,9674000000,NaN,3483000000,...,306849526,306000000,309000000,5.059,5269000000,0,24000000,0,17.219,1796000000
4,A,MRQ,2020-09-30,2020-10-31,2020-10-31,2021-03-02,-522000000,9627000000,NaN,3415000000,...,308309635,309000000,312000000,4.799,5194000000,0,61000000,0,16.809,1948000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,ZYXI,ARY,2020-12-31,2021-02-25,2020-12-31,2021-02-26,0,72175000,51780250.0,63023000,...,34849482,33869000,34943000,2.366,72175000,566000,1079000,280000,2.131,52906000
6,ZYXI,ART,2020-12-31,2021-02-25,2020-12-31,2021-02-26,0,72175000,51780250.0,63023000,...,34849482,33869000,34943000,2.366,72175000,566000,1079000,280000,2.131,52906000
7,ZYXI,ART,2020-09-30,2020-10-27,2020-09-30,2021-02-26,0,64965000,40805750.0,57553000,...,34740990,34486000,35476000,1.992,64965000,985000,1429000,429000,1.884,50258000


In [19]:
sf1 = sf1[sf1.dimension == "ART"]
sf1

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
6,A,ART,2020-12-31,2021-03-02,2021-01-31,2021-03-02,-469000000,9674000000,9575500000.0,3483000000,...,304697515,306000000,309000000,18.072,5269000000,0,125000000,0,17.219,1796000000
7,A,ART,2020-09-30,2020-12-18,2020-10-31,2021-03-02,-522000000,9627000000,9532250000.0,3415000000,...,306849526,309000000,312000000,17.278,5194000000,0,123000000,0,16.809,1948000000
6,AA,ART,2020-12-31,2021-02-25,2020-12-31,2021-02-25,-5629000000,14860000000,14015250000.0,4520000000,...,186251518,186251518,None,49.857,14860000000,655000000,187000000,300000000,79.785,1759000000
7,AA,ART,2020-09-30,2020-10-30,2020-09-30,2021-02-25,-5547000000,14250000000,13958000000.0,4053000000,...,185929586,185929586,None,50.180,14250000000,540000000,221000000,301000000,76.642,1750000000
6,AAL,ART,2020-12-31,2021-02-17,2020-12-31,2021-02-17,-7103000000,62008000000,61976250000.0,11095000000,...,639675800,639675800,None,27.103,55888000000,3239000000,-2568000000,0,87.369,-5474000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,ZYME,ART,2020-09-30,2020-11-03,2020-09-30,2021-02-24,-6659000,571719000,545838750.0,487320000,...,45829382,50903633,50903633,0.494,553306000,1745000,-234000,0,10.870,390428000
6,ZYNE,ART,2020-12-31,2021-03-10,2020-12-31,2021-03-10,0,74056776,78802650.0,73366174,...,41251537,27022931,27022931,0.000,74056776,9042586,0,0,2.741,59452926
7,ZYNE,ART,2020-09-30,2020-11-09,2020-09-30,2021-03-10,0,77205413,82229606.0,74757396,...,29439175,29243375,29243375,0.000,77205413,8217695,0,0,2.640,64191541


In [20]:
sf1.reset_index(inplace=True)

In [21]:
sep = pd.DataFrame()

In [22]:
for tick in final_ticks:
    sep_ticker = quandl.get_table('SHARADAR/SEP', ticker=tick, date=dates)
    sep = sep.append(sep_ticker)

In [23]:
sep

,ticker,date,open,high,low,close,volume,dividends,closeunadj,lastupdated
None,,,,,,,,,,
0,A,2021-03-16,124.75,125.305,122.450,123.00,1345485.0,0.0,123.00,2021-03-16
1,A,2021-03-15,120.92,124.280,120.340,124.04,1229988.0,0.0,124.04,2021-03-15
2,A,2021-03-12,120.57,122.030,119.280,121.44,1368747.0,0.0,121.44,2021-03-12
3,A,2021-03-11,121.42,122.920,120.900,121.83,1818696.0,0.0,121.83,2021-03-11
4,A,2021-03-10,120.59,122.210,119.685,119.99,1934557.0,0.0,119.99,2021-03-10
...,...,...,...,...,...,...,...,...,...,...
7,ZYXI,2021-03-05,14.65,14.887,13.804,14.75,1161003.0,0.0,14.75,2021-03-06
8,ZYXI,2021-03-04,14.25,14.570,13.703,14.54,1549927.0,0.0,14.54,2021-03-04
9,ZYXI,2021-03-03,14.85,14.850,13.710,14.25,1009483.0,0.0,14.25,2021-03-03


In [ ]:
def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [ ]:
dcfs = []

In [ ]:
for ticker in final_tickers:
    url = ("https://financialmodelingprep.com/api/v3/historical-daily-discounted-cash-flow/"+ticker+"?limit=30&apikey=8c48192b57789b1b85a59db736780f87")
    dcf = get_jsonparsed_data(url)
    dcfs.append(dcf)

In [ ]:
from itertools import chain
dcf = pd.DataFrame(list(chain.from_iterable(dcfs)))
dcf

In [ ]:
surprises = []

In [ ]:
for ticker in final_tickers:
    url = ("https://financialmodelingprep.com/api/v3/earnings-surpises/"+ticker+"?apikey=8c48192b57789b1b85a59db736780f87")
    surprise = get_jsonparsed_data(url)
    surprises.append(surprise)

In [ ]:
from itertools import chain
surprise = pd.DataFrame(list(chain.from_iterable(surprises)))
surprise

In [129]:
final_df = pd.read_pickle('final_df.pkl')

In [130]:
final_df

,ticker,date,close,volume,calendardate,accoci,assets,assetsavg,assetsc,assetsnc,...,rating_cnt_mod_sells,rating_cnt_strong_sells,rating_mean_recom,rating_cnt_with,rating_cnt_without,rating_change,dcf,actualEarningResult,estimatedEarning,earnings_surprise
0,A,2019-02-20,78.56,2076523.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,1.1,10.0,0.0,0,84.948199,0.76,0.73,0.03
1,A,2019-02-21,77.89,3570828.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,1.1,10.0,0.0,0,84.278199,0.76,0.73,0.03
2,A,2019-02-22,78.42,2797669.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,1.1,10.0,0.0,1,84.808199,0.76,0.73,0.03
3,A,2019-02-25,79.34,1878041.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,1.1,10.0,0.0,0,85.728199,0.76,0.73,0.03
4,A,2019-02-26,78.55,1992595.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,1.1,10.0,0.0,0,84.938199,0.76,0.73,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081588,ZYXI,2021-02-23,18.07,646106.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,2.0,6.0,0.0,0,19.232166,0.04,0.05,-0.01
1081589,ZYXI,2021-02-24,18.42,315802.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,2.0,6.0,0.0,0,19.582166,0.04,0.05,-0.01
1081590,ZYXI,2021-02-25,17.42,388872.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,2.3,6.0,0.0,1,18.524211,0.05,0.06,-0.01
1081591,ZYXI,2021-02-26,14.52,2337972.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,2.3,6.0,0.0,0,15.624211,0.05,0.06,-0.01


In [131]:
one_hot_features = ['exchange', 'sector', 'industry']

for feature in one_hot_features:
    cat_X = final_df.loc[:, [feature]]
    ohe = OneHotEncoder(drop='first', sparse=False)
    ohe.fit(cat_X)
    ohe_X = ohe.transform(cat_X)
    columns = ohe.get_feature_names([feature])
    ohe_X_df = pd.DataFrame(ohe_X, columns=columns, index=cat_X.index)
    final_df = final_df.join(ohe_X_df)

In [132]:
final_df

,ticker,date,close,volume,calendardate,accoci,assets,assetsavg,assetsc,assetsnc,...,industry_Travel Services,industry_Trucking,industry_Uranium,industry_Utilities - Diversified,industry_Utilities - Independent Power Producers,industry_Utilities - Regulated Electric,industry_Utilities - Regulated Gas,industry_Utilities - Regulated Water,industry_Utilities - Renewable,industry_Waste Management
0,A,2019-02-20,78.56,2076523.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,2019-02-21,77.89,3570828.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,2019-02-22,78.42,2797669.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A,2019-02-25,79.34,1878041.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A,2019-02-26,78.55,1992595.0,2018-12-31,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081588,ZYXI,2021-02-23,18.07,646106.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081589,ZYXI,2021-02-24,18.42,315802.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081590,ZYXI,2021-02-25,17.42,388872.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081591,ZYXI,2021-02-26,14.52,2337972.0,2020-12-31,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
final_df.to_pickle("streamlit_df.pkl")